In [3]:
import polars as pl
import pyarrow.parquet as pq
import sys

filepath = "../Data/processed/sirene_infos_FINAL_DuelML.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    print(f"Lecture du fichier via PyArrow : {filepath}")
    table_arrow = pq.read_table(
        filepath,
    )
    
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_bilan = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---\n")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_bilan.head())

except Exception as e:
    print(f"\n--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow : ../Data/processed/sirene_infos_FINAL_DuelML.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---

Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ date_clot ┆ CJCK_Tota ┆ HN_Résult ┆ … ┆ ratio_tre ┆ anciennet ┆ cible_Res ┆ cible_HN │
│ ---       ┆ ure_exerc ┆ lActifBru ┆ atNet     ┆   ┆ sorerie   ┆ e_entrepr ┆ ultatNet_ ┆ _Résulta │
│ str       ┆ ice       ┆ t         ┆ ---       ┆   ┆ ---       ┆ ise       ┆ T_plus_1  ┆ tNet_T_p │
│           ┆ ---       ┆ ---       ┆ i32       ┆   ┆ f64       ┆ ---       ┆ ---       ┆ lus_1    │
│           ┆ date      ┆ i32       ┆           ┆   ┆           ┆ i32       ┆ i32       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ i32      │
╞═══════════╪═

In [4]:
import polars as pl
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# =========================
# 1. DATA PREP
# =========================
df = df_bilan.to_pandas()

# enlève les trois premières colonnes (ID, année, etc.)
df = df.iloc[:, 3:]

X = df.drop('cible_ResultatNet_T_plus_1', axis=1)
y = df['cible_ResultatNet_T_plus_1']

# Train / test split (test kept untouched for final evaluation)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# =========================
# 2. BASELINE MODEL
# =========================
print(f"\n[BASELINE] Entraînement sur {len(X_train)} entreprises...")

baseline_model = xgb.XGBRegressor(
    objective='reg:absoluteerror',  # optimise MAE
    random_state=42,
    n_jobs=-1
)

baseline_model.fit(X_train, y_train)
baseline_pred = baseline_model.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_r2 = r2_score(y_test, baseline_pred)

print("\n--- PERFORMANCE BASELINE ---")
print(f"MAE : {baseline_mae:,.2f} €")
print(f"R²  : {baseline_r2:.4f}")

# =========================
# 3. HYPERPARAMETER SEARCH (RandomizedSearchCV)
# =========================
print("\n[SEARCH] Optimisation des hyperparamètres avec RandomizedSearchCV...")

param_dist = {
    "n_estimators": [300, 600, 900, 1200, 1500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "max_depth": [3, 5, 7, 9],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "min_child_weight": [1, 3, 5, 10],
    "gamma": [0, 0.1, 0.3, 1.0],
    "reg_alpha": [0, 0.1, 1.0],
    "reg_lambda": [0.5, 1.0, 2.0],
}

search_model = xgb.XGBRegressor(
    objective='reg:absoluteerror',
    random_state=42,
    n_jobs=-1
)

random_search = RandomizedSearchCV(
    estimator=search_model,
    param_distributions=param_dist,
    n_iter=10,                    # nombre d'essais (augmentable si tu veux)
    scoring='neg_mean_absolute_error',
    cv=5,                         # 5-fold CV (exigence du cours)
    verbose=1,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

print("\nMeilleurs hyperparamètres trouvés :")
print(random_search.best_params_)
print(f"MAE CV (val) ≈ {-random_search.best_score_:,.2f} €")

# =========================
# 4. FINAL MODEL WITH EARLY STOPPING
# =========================
print("\n[FINAL] Ré-entraînement du meilleur modèle avec early stopping...")

best_params = random_search.best_params_

# On refait un split interne pour early stopping (train / valid)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

final_model = xgb.XGBRegressor(
    **best_params,
    objective='reg:absoluteerror',
    random_state=42,
    n_jobs=-1
)

final_model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=50,
    verbose=50
)

# ==========================================
# 5. RÉSULTATS SUR LE TEST SET
# ==========================================
predictions = final_model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("\n--- PERFORMANCE MODÈLE OPTIMISÉ ---")
print(f"MAE : {mae:,.2f} € (baseline : {baseline_mae:,.2f} €)")
print(f"R²  : {r2:.4f} (baseline : {baseline_r2:.4f})")

# =========================
# 6. FEATURE IMPORTANCE
# =========================
plt.figure(figsize=(12, 8))
xgb.plot_importance(final_model, max_num_features=20, height=0.5,
                    title="Variables les plus influentes sur le Résultat Net N+1")
plt.tight_layout()
plt.savefig("feature_importance.png")

# =========================
# 7. PLOT PRÉDICTION vs RÉALITÉ
# =========================
plt.figure(figsize=(10, 10))
plt.scatter(y_test, predictions, alpha=0.4, s=10)
p1 = max(max(predictions), max(y_test))
p0 = min(min(predictions), min(y_test))
plt.plot([p0, p1], [p0, p1], 'r--')
plt.xlabel('Vrai Résultat Net N+1')
plt.ylabel('Résultat Prédit N+1')
plt.title('Précision du Modèle XGBoost (optimisé)')
plt.xscale('symlog')
plt.yscale('symlog')
plt.tight_layout()
plt.savefig("prediction_vs_realite.png")

print("\nGraphiques générés : 'feature_importance.png' et 'prediction_vs_realite.png'")

(195822, 8) (48956, 8) (195822,) (48956,)

[BASELINE] Entraînement sur 195822 entreprises...

--- PERFORMANCE BASELINE ---
MAE : 176,894.16 €
R²  : 0.5943

[SEARCH] Optimisation des hyperparamètres avec RandomizedSearchCV...
Fitting 5 folds for each of 10 candidates, totalling 50 fits

Meilleurs hyperparamètres trouvés :
{'subsample': 0.8, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 900, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.02, 'gamma': 0.3, 'colsample_bytree': 0.8}
MAE CV (val) ≈ 177,365.00 €

[FINAL] Ré-entraînement du meilleur modèle avec early stopping...


TypeError: XGBModel.fit() got an unexpected keyword argument 'early_stopping_rounds'